# CONFLUENCE Tutorial: Regional Domain Modeling - Iceland

This notebook demonstrates regional domain modeling using Iceland as an example. Unlike previous tutorials that focused on single watersheds, this example:

1. **Delineates an entire region** (not just a single watershed)
2. **Includes coastal watersheds** (watersheds that drain directly to the ocean)


## Learning Objectives

1. Understand regional vs. watershed-scale modeling
2. Learn how to delineate coastal watersheds
3. Handle multiple independent drainage systems
4. Work with national/regional scale domains

## 1. Setup

In [1]:
# Import required libraries
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from shapely.geometry import box
import contextily as cx
from datetime import datetime
import xarray as xr
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import CONFLUENCE
from CONFLUENCE import CONFLUENCE

plt.style.use('default')
%matplotlib inline

print(f"Working from: {confluence_path}")

Working from: /home/darri.eythorsson/code/CONFLUENCE


## 2. Initialize CONFLUENCE
First, let's set up our directories and load the configuration. We'll customize the Iceland configuration file.

In [2]:
# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/work/comphyd_lab/data/CONFLUENCE_data')  # ← User should modify this path

# Load Iceland configuration
iceland_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_Iceland.yaml'
with open(iceland_config_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Create tutorial version with updated domain name
config_dict['DOMAIN_NAME'] = "Iceland_tutorial"  
config_dict['EXPERIMENT_ID'] = "tutorial_run_1"
config_dict['EXPERIMENT_TIME_START'] = '2011-01-01 01:00'
config_dict['EXPERIMENT_TIME_END'] = '2022-12-31 23:00'

# Create config directory if it doesn't exist
config_dir = CONFLUENCE_CODE_DIR / '0_config_files'
config_dir.mkdir(parents=True, exist_ok=True)

# Write to tutorial config file
tutorial_config_path = config_dir / 'config_iceland_tutorial.yaml'
with open(tutorial_config_path, 'w') as f:
    yaml.dump(config_dict, f)

try:
    # Initialize CONFLUENCE with tutorial config
    confluence = CONFLUENCE(tutorial_config_path)
    
    # Parse bounding box for visualization
    bbox = config_dict['BOUNDING_BOX_COORDS'].split('/')
    lat_max, lon_min, lat_min, lon_max = map(float, bbox)
    
    # Display configuration
    print("=== Iceland Tutorial Configuration ===")
    print(f"Domain Name: {confluence.config['DOMAIN_NAME']}")
    print(f"Bounding Box: {confluence.config['BOUNDING_BOX_COORDS']}")
    print(f"Delineate by Pour Point: {confluence.config['DELINEATE_BY_POURPOINT']} (Full region!)")
    print(f"Include Coastal Watersheds: {confluence.config.get('DELINEATE_COASTAL_WATERSHEDS', True)}")
    print(f"Stream Threshold: {confluence.config['STREAM_THRESHOLD']}")
    print(f"Domain Method: {confluence.config['DOMAIN_DEFINITION_METHOD']}")
    
except Exception as e:
    print(f"Error initializing CONFLUENCE: {str(e)}")
    print("Check that CONFLUENCE_DATA_DIR is correctly set and accessible.")

19:22:28 - INFO - ============================================================


2025-05-13 19:22:28,634 - confluence_general - INFO - ============================================================


19:22:28 - INFO - CONFLUENCE Logging Initialized


2025-05-13 19:22:28,636 - confluence_general - INFO - CONFLUENCE Logging Initialized


19:22:28 - INFO - Domain: Iceland_tutorial


...

19:22:28 - INFO - Initializing VariableHandler for dataset: ERA5 and model: SUMMA


2025-05-13 19:22:28,709 - confluence_general - INFO - Initializing VariableHandler for dataset: ERA5 and model: SUMMA


19:22:29 - INFO - CONFLUENCE system initialized successfully


2025-05-13 19:22:29,052 - confluence_general - INFO - CONFLUENCE system initialized successfully


=== Iceland Tutorial Configuration ===
Domain Name: Iceland_tutorial
Bounding Box: 66.8/-25.5/63.0/-13.0
Delineate by Pour Point: False (Full region!)
Include Coastal Watersheds: True
Stream Threshold: 5000
Domain Method: delineate


## 4. Project Setup
The first step is to set up our project structure. Since we're doing a regional model, we need a different approach than for a single watershed.

In [ ]:
# Step 1: Project Initialization
print("=== Step 1: Project Initialization ===")

try:
    # Setup project
    project_dir = confluence.managers['project'].setup_project()    
    pour_point_path = confluence.managers['project'].create_pour_point()

    # List created directories
    print("\nCreated directories:")
    created_dirs = []
    for item in sorted(project_dir.iterdir()):
        if item.is_dir():
            created_dirs.append(item.name)
            print(f"  📁 {item.name}")
    
    # Check if all required directories are created
    required_dirs = ['shapefiles', 'attributes', 'forcing', 'simulations', 'evaluation', 'plots']
    for dir_name in required_dirs:
        if dir_name not in created_dirs:
            print(f"Warning: Required directory '{dir_name}' not created")
    
    print("\nDirectory purposes:")
    print("  📁 shapefiles: Domain geometry (multiple watersheds, river network)")
    print("  📁 attributes: Static characteristics (elevation, soil, land cover)")
    print("  📁 forcing: Meteorological inputs (precipitation, temperature)")
    print("  📁 simulations: Model outputs")
    print("  📁 evaluation: Performance metrics and comparisons")
    print("  📁 plots: Visualizations")
except Exception as e:
    print(f"Error setting up project: {str(e)}")

## 5. Geospatial Domain Definition and Analysis - Data Acquisition
Before delineating the region, we need to acquire geospatial data (DEM, soil, land cover).

In [ ]:
# Step 2: Geospatial Domain Definition and Analysis
print("=== Step 2: Geospatial Domain Definition and Analysis ===")

try:
    # Acquire attributes
    print("Acquiring geospatial attributes (DEM, soil, land cover)...")
    confluence.managers['data'].acquire_attributes()
        
except Exception as e:
    print(f"Error acquiring attributes: {str(e)}")

## 6. Regional Domain Delineation
This is the critical step where we delineate the entire region, including coastal watersheds. This is different from the single-watershed approach.

In [ ]:
try:
    # Define domain
    print(f"Delineating regional domain using method: {confluence.config['DOMAIN_DEFINITION_METHOD']}")
    print(f"Delineate by pour point: {confluence.config['DELINEATE_BY_POURPOINT']} (Full region!)")
    print(f"Include coastal watersheds: {confluence.config.get('DELINEATE_COASTAL_WATERSHEDS', True)}")
    print(f"Stream threshold: {confluence.config['STREAM_THRESHOLD']}")
    print("\nThis will create multiple independent drainage basins...")
    
    watershed_path = confluence.managers['domain'].define_domain()
    
    # Check results
    basin_path = project_dir / 'shapefiles' / 'river_basins'
    network_path = project_dir / 'shapefiles' / 'river_network'
    
    basin_count = 0
    basin_files = []
    basins = None
    if basin_path.exists():
        basin_files = list(basin_path.glob('*.shp'))
        if basin_files:
            try:
                basins = gpd.read_file(basin_files[0])
                basin_count = len(basins)
                print(f"\n✓ Created {basin_count} watersheds")
                print(f"Total area: {basins.geometry.area.sum() / 1e6:.0f} km²")
            except Exception as e:
                print(f"Error reading basin shapefile: {str(e)}")
    
    network_count = 0
    network_files = []
    rivers = None
    if network_path.exists():
        network_files = list(network_path.glob('*.shp'))
        if network_files:
            try:
                rivers = gpd.read_file(network_files[0])
                network_count = len(rivers)
                print(f"✓ Created river network with {network_count} segments")
            except Exception as e:
                print(f"Error reading river network shapefile: {str(e)}")
                
    if not basin_files:
        print("⚠ No basin shapefiles found. Domain delineation may have failed.")
    if not network_files:
        print("⚠ No river network shapefiles found. Stream delineation may have failed.")
        
except Exception as e:
    print(f"Error during domain delineation: {str(e)}")

## 7. Watershed Discretization
Now we need to discretize our domain into GRUs (Grouped Response Units) and HRUs (Hydrologic Response Units).

In [ ]:
try:
    # Discretize domain
    print(f"Creating HRUs using method: {confluence.config['DOMAIN_DISCRETIZATION']}")
    hru_path = confluence.managers['domain'].discretize_domain()
        
except Exception as e:
    print(f"Error during domain discretization: {str(e)}")

## 8. Visualize Regional Domain
Let's visualize what our regional domain looks like with all delineated watersheds.

In [ ]:
try:
    # Create CONFLUENCE domain visualization
    print("Creating regional domain visualization...")
    if hasattr(confluence.managers['domain'], 'plot_domain'):
        plot_paths = confluence.managers['domain'].plot_domain()
    else:
        print("plot_domain method not available - using custom visualization instead")
    
    # Create custom visualization
    if basin_path.exists() and basin_files and basins is not None:
        fig, ax = plt.subplots(figsize=(14, 10))
        
        # Plot watersheds
        if 'GRU_ID' in basins.columns:
            basins.plot(ax=ax, column='GRU_ID', cmap='tab20', 
                       edgecolor='black', linewidth=0.5, legend=False)
        else:
            basins.plot(ax=ax, cmap='tab20', 
                       edgecolor='black', linewidth=0.5, legend=False)
        
        # Plot river network if available
        if network_path.exists() and network_files and rivers is not None:
            rivers.plot(ax=ax, color='blue', linewidth=1)
        
        ax.set_title(f'Iceland Regional Domain - {basin_count} Watersheds', 
                    fontsize=16, fontweight='bold')
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        
        # Add annotation about coastal watersheds
        ax.text(0.02, 0.98, f'Including coastal watersheds\nTotal watersheds: {basin_count}',
                transform=ax.transAxes, va='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                fontsize=12)
        
        plt.tight_layout()
        plt.show()
    else:
        print("Cannot create visualization: Basin data not available")
except Exception as e:
    print(f"Error creating domain visualization: {str(e)}")

## Acquire forcing data

In [ ]:
# Acquire forcings
print(f"\nAcquiring forcing data: {confluence.config['FORCING_DATASET']}")
confluence.managers['data'].acquire_forcings()

## Model Agnostic Preprocessing 

In [6]:
# Run model-agnostic preprocessing
print("\nRunning model-agnostic preprocessing...")
confluence.managers['data'].run_model_agnostic_preprocessing()


Running model-agnostic preprocessing...
18:12:53 - INFO - Starting model-agnostic preprocessing


2025-05-13 18:12:53,937 - confluence_general - INFO - Starting model-agnostic preprocessing


18:12:53 - INFO - Running geospatial statistics


2025-05-13 18:12:53,964 - confluence_general - INFO - Running geospatial statistics


18:12:53 - INFO - Starting geospatial statistics calculation


...

2025-05-13 18:12:58,698 - confluence_general - INFO - Forcing remapping process completed


18:12:58 - INFO - Forcing data resampling process completed


2025-05-13 18:12:58,699 - confluence_general - INFO - Forcing data resampling process completed


18:12:58 - INFO - Model-agnostic preprocessing completed successfully


2025-05-13 18:12:58,702 - confluence_general - INFO - Model-agnostic preprocessing completed successfully


## Model Specific Preprocessing

In [7]:
# Step 4: Model Specific Processing and Initialization
print("=== Step 4: Model Specific Processing and Initialization ===")

# Preprocess models
print(f"Preparing {confluence.config['HYDROLOGICAL_MODEL']} input files...")
confluence.managers['model'].preprocess_models()

=== Step 4: Model Specific Processing and Initialization ===
Preparing SUMMA input files...
18:12:58 - INFO - Starting model-specific preprocessing


2025-05-13 18:12:58,709 - confluence_general - INFO - Starting model-specific preprocessing


18:12:58 - INFO - Processing model: SUMMA


2025-05-13 18:12:58,718 - confluence_general - INFO - Processing model: SUMMA


18:12:58 - INFO - Starting SUMMA spatial preprocessing


...

2025-05-13 18:32:49,719 - confluence_general - INFO - mizuRoute control file created at /work/comphyd_lab/data/CONFLUENCE_data/domain_Iceland_tutorial/settings/mizuRoute/mizuroute.control


18:32:49 - INFO - mizuRoute spatial preprocessing completed


2025-05-13 18:32:49,721 - confluence_general - INFO - mizuRoute spatial preprocessing completed


18:32:49 - INFO - Model-specific preprocessing completed


2025-05-13 18:32:49,722 - confluence_general - INFO - Model-specific preprocessing completed


## Model initiation

In [ ]:
# Run models
print(f"\nRunning {confluence.config['HYDROLOGICAL_MODEL']} model...")
confluence.managers['model'].run_models()

print("\nModel run complete")


Running SUMMA model...
19:22:29 - INFO - Starting model runs


2025-05-13 19:22:29,671 - confluence_general - INFO - Starting model runs


19:22:29 - INFO - Running model: SUMMA


2025-05-13 19:22:29,674 - confluence_general - INFO - Running model: SUMMA


19:22:29 - INFO - Starting SUMMA run


2025-05-13 19:22:29,676 - confluence_general - INFO - Starting SUMMA run
